# **Programming Assessment \#4**

Names: Enriquez, Manolo L.  Narvaez, Jose Wilfredo S.

More information on the assessment is found in our Canvas course. Link: https://dlsu.instructure.com/courses/93383/assignments/739602

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [8]:
import nltk
from collections import Counter
nltk.download('gutenberg')

# Language Model
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
  words = [word.lower() for word in nltk.corpus.gutenberg.words(filename)]
  all_words.update(words)

total_tokens = sum(all_words.values())
total_words = len(all_words)

# Error Model class
class ErrorModel:
    def __init__(self, character, correction, occurence):
        self.character  = character
        self.correction = correction
        self.occurence = occurence

# Loading error model text file into dictionary
error_model = {}
file1 = open('count_1edit.txt', 'r')
Lines = file1.readlines()

for line in Lines:
    key = line.split('\t')[0]
    value = int(line.split('\t')[1])
    error_model[key] = value

print(error_model[' |-'])



[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\jolon\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


102


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below. And while we discussed the general approach in class, kindly describe how you decided to implement the spell correction model. Include any modifications your group made as well. This might be a good spot to place part of your write up.*

In [3]:
word = input("Input: ")
# print("Output: " + str(all_words[word]))

one_edit_list = []

if word not in all_words:
    # Look for words with one edit distance in language model
    for key in all_words:
        edit_distance = nltk.edit_distance(word, key, substitution_cost=1, transpositions=True)
        if edit_distance == 1:
            one_edit_list.append(key)
            
    for candidate in one_edit_list:
        p_word = all_words[candidate] / total_words
        print(candidate, end=" ")
        print(p_word, end=" ")
        print(nltk.edit_distance_align(word, candidate, substitution_cost=1))
else:
    print("Output: No error")





Input: nogsy
noisy 0.00040152105623656673 [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]


# **Your Relfection / Takeaway / Analysis**

*Kindly place the rest of your write up. More information is found in the Canvas write up.*